<a href="https://colab.research.google.com/github/mariaaslam706/CVD_data_XQFD_demo/blob/main/CVD_XQFD_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#All Libraries
!pip install explainerdashboard
!pip install graphviz

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from explainerdashboard import RegressionExplainer, ExplainerDashboard

!pip install dalex
import dalex as dx

import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

! pip install plotly
import plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
#Load data

df = pd.read_csv(r"/content/heart_failure_2.csv")
print (df.shape)
print (df.columns.values)

#count values in target column
df.head()

In [ ]:
#Seperate the labels
X= df.drop(['DEATH_EVENT'], axis=1)
Y = df['DEATH_EVENT']

In [ ]:
#train and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(209, 11) (209,) (90, 11) (90,)


In [ ]:
#Training the model
model = RandomForestRegressor(n_estimators=500, random_state = 42)
model.fit(X_train, Y_train.values.ravel())

RandomForestRegressor(n_estimators=500, random_state=42)

In [ ]:
#######Dashboard XAI#######
explainer = RegressionExplainer(model, X_test, Y_test)
#Start the Dashboard
db = ExplainerDashboard(explainer,whatif=False)
#Running the app on a local port 3050
db.run(port=8050)

Changing class type to RandomForestRegressionExplainer...
Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected google colab environment, setting mode='external'
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating shap interaction values...
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating dependencies...
Calculating importances...
Calculating ShadowDecTree for each individual decision tree...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...
Starting ExplainerDashboard on http://172.28.0.12:8050
You can terminate

<IPython.core.display.Javascript object>

In [ ]:
ExplainerDashboard.terminate(3050)

In [ ]:
#######Dalex XAI#######
exp = dx.Explainer(model,X_train,Y_train)
#Model performance metrix
exp.model_performance()

Preparation of a new explainer is initiated

  -> data              : 209 rows 11 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 209 values
  -> model_class       : sklearn.ensemble._forest.RandomForestRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x7999c2becaf0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.335, max = 0.984
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.368, mean = -0.000498, max = 0.386
  -> model_info        : package sklearn

A new explainer has been created!


,mse,rmse,r2,mae,mad
RandomForestRegressor,0.025093,0.158407,0.887351,0.124153,0.102


In [ ]:
exp.model_parts()

In [ ]:
exp.model_parts().plot()

In [ ]:
data_sample = X_test.iloc[7]
data_sample

In [ ]:
Y_test.iloc[7]
print("RFR:",model.predict([data_sample]))

RFR: [0.02]


In [ ]:
data_frame = pd.DataFrame(data_sample).T

In [ ]:
explanation = exp.predict_parts(data_frame)
explanation.result

,variable_name,variable_value,variable,cumulative,contribution,sign,position,label
0,intercept,,intercept,0.335426,0.335426,1.0,12,RandomForestRegressor
1,serum_creatinine,1.1,serum_creatinine = 1.1,0.277962,-0.057464,-1.0,11,RandomForestRegressor
2,serum_sodium,139.0,serum_sodium = 139.0,0.248670,-0.029292,-1.0,10,RandomForestRegressor
3,ejection_fraction,35.0,ejection_fraction = 35.0,0.161340,-0.087330,-1.0,9,RandomForestRegressor
4,age,55.0,age = 55.0,0.089895,-0.071445,-1.0,8,RandomForestRegressor
5,platelets,254000.0,platelets = 254000.0,0.054048,-0.035847,-1.0,7,RandomForestRegressor
6,anaemia,0.0,anaemia = 0.0,0.034507,-0.019541,-1.0,6,RandomForestRegressor
7,high_blood_pressure,0.0,high_blood_pressure = 0.0,0.030239,-0.004268,-1.0,5,RandomForestRegressor
8,creatinine_phosphokinase,109.0,creatinine_phosphokinase = 109.0,0.021895,-0.008344,-1.0,4,RandomForestRegressor
9,sex,1.0,sex = 1.0,0.020000,-0.001895,-1.0,3,RandomForestRegressor


In [ ]:
explanation.plot()

In [ ]:
john = pd.DataFrame({'age': [55],
                       'anaemia': [1],
                       'creatinine_phosphokinase': [850],
                       'diabetes': [1],
                       'ejection_fraction': [72],
                       'high_blood_pressure': [1],
                       'platelets': 22000 ,
                       'serum_creatinine': [1.3] ,
                       'serum_sodium': [172] ,
                       'sex': [1] ,
                       'smoking':[0]},
                      index = ['John'])

In [ ]:
exp.predict(X)[0:10]

array([0.952, 0.678, 0.918, 0.862, 0.652, 0.546, 0.926, 0.774, 0.636,
       0.89 ])

In [ ]:
exp.predict(john)

array([0.48])

In [ ]:
Maria = pd.DataFrame({'age': [20],
                       'anaemia': [0],
                       'creatinine_phosphokinase': [0],
                       'diabetes': [0],
                       'ejection_fraction': [30],
                       'high_blood_pressure': [0],
                       'platelets': 0 ,
                       'serum_creatinine': [14] ,
                       'serum_sodium': [0] ,
                       'sex': [1] ,
                       'smoking':[1]},
                      index = ['Maria'])

In [ ]:
exp.predict(X)[0:10]

array([0.952, 0.678, 0.918, 0.862, 0.652, 0.546, 0.926, 0.774, 0.636,
       0.89 ])

In [ ]:
exp.predict(Maria)

array([0.312])

In [ ]:
bd_john = exp.predict_parts(john, type='break_down', label=john.index[0])
bd_interactions_john = exp.predict_parts(john, type='break_down_interactions', label="John+")

sh_Maria = exp.predict_parts(Maria, type='shap', B=10, label=Maria.index[0])

In [ ]:
bd_john.result

,variable_name,variable_value,variable,cumulative,contribution,sign,position,label
0,intercept,,intercept,0.335426,0.335426,1.0,12,John
1,platelets,22000.0,platelets = 22000.0,0.406297,0.070871,1.0,11,John
2,serum_sodium,172.0,serum_sodium = 172.0,0.472048,0.065751,1.0,10,John
3,ejection_fraction,72.0,ejection_fraction = 72.0,0.486727,0.014679,1.0,9,John
4,anaemia,1.0,anaemia = 1.0,0.511081,0.024354,1.0,8,John
5,high_blood_pressure,1.0,high_blood_pressure = 1.0,0.514163,0.003081,1.0,7,John
6,diabetes,1.0,diabetes = 1.0,0.521541,0.007378,1.0,6,John
7,smoking,0.0,smoking = 0.0,0.525225,0.003684,1.0,5,John
8,sex,1.0,sex = 1.0,0.494220,-0.031005,-1.0,4,John
9,creatinine_phosphokinase,850.0,creatinine_phosphokinase = 850.0,0.546612,0.052392,1.0,3,John


In [ ]:
bd_john.plot(bd_interactions_john)

In [ ]:
sh_Maria.result.loc[sh_Maria.result.B == 0, ]

,variable,contribution,variable_name,variable_value,sign,label,B
0,creatinine_phosphokinase = 0.0,-0.069731,creatinine_phosphokinase,0,-1.0,Maria,0
1,age = 20.0,-0.069035,age,20,-1.0,Maria,0
2,serum_sodium = 0.0,0.062370,serum_sodium,0,1.0,Maria,0
3,platelets = 0.0,0.041704,platelets,0,1.0,Maria,0
4,serum_creatinine = 14.0,0.022407,serum_creatinine,14,1.0,Maria,0
5,sex = 1.0,-0.008211,sex,1,-1.0,Maria,0
6,anaemia = 0.0,-0.005707,anaemia,0,-1.0,Maria,0
7,ejection_fraction = 30.0,0.005599,ejection_fraction,30,1.0,Maria,0
8,high_blood_pressure = 0.0,-0.004308,high_blood_pressure,0,-1.0,Maria,0
9,smoking = 0.0,0.003051,smoking,0,1.0,Maria,0


In [ ]:
sh_Maria.plot(bar_width=16)

In [ ]:
exp.predict_parts(john, type='shap', B=10, label=john.index[0]).plot(max_vars=5)

In [ ]:
mp = exp.model_performance(model_type = 'regression')
mp.result

,mse,rmse,r2,mae,mad
RandomForestRegressor,0.025093,0.158407,0.887351,0.124153,0.102


In [ ]:
cp_Maria = exp.predict_profile(Maria, label=Maria.index[0])
cp_john = exp.predict_profile(john, label=john.index[0])

cp_Maria.result.head()

Calculating ceteris paribus: 100%|██████████| 11/11 [00:00<00:00, 27.52it/s]


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,_original_,_yhat_,_vname_,_ids_,_label_
Maria,20.00,0.0,0.0,0.0,30.0,0.0,0,14.0,0.0,1.0,0.0,20,0.312,age,Maria,Maria
Maria,40.00,0.0,0.0,0.0,30.0,0.0,0,14.0,0.0,1.0,0.0,20,0.312,age,Maria,Maria
Maria,40.55,0.0,0.0,0.0,30.0,0.0,0,14.0,0.0,1.0,0.0,20,0.312,age,Maria,Maria
Maria,41.10,0.0,0.0,0.0,30.0,0.0,0,14.0,0.0,1.0,0.0,20,0.312,age,Maria,Maria
Maria,41.65,0.0,0.0,0.0,30.0,0.0,0,14.0,0.0,1.0,0.0,20,0.312,age,Maria,Maria
